# Universal dependencies

For my purposes, universal dependencies is a way of hand-annotating many languages with a consistent set of features. The data comes in a very nice [format](http://universaldependencies.org/docs/format.html).

In [10]:
import os
import re
import numpy as np
import pandas as pd

In [11]:
def read(path):
    with open(path) as f:
        return f.read()

Every sentence has comment lines at the top, indicated with a #.

In [12]:
def remove_comments(raw):
    comment_pattern = '# .*?\n' # Must include space after hashtag so that hashtag tokens are parsed correctly
    return re.sub(comment_pattern, '', raw)

In [13]:
def write(path, data):
    with open(path, 'w') as f:
        f.write(data)

In [14]:
train_data_path = 'UD_English/en-ud-train.conllu'
raw = read(train_data_path)
cleaned = remove_comments(raw)
path = 'UD_English/en-ud-train-cleaned.tsv'
write(path, cleaned)

In [15]:
columns = ['id', 'form', 'lemma', 'universal_pos', 'lg_pos', 'features', 'head', 'dependency_relations', 'dependencies', 'misc']
df = pd.read_csv(path, sep='\t', names=columns, quoting=3)
df['id'] = df['id'].astype(int)
df.replace('_', np.nan, inplace=True)
df.head()

,id,form,lemma,universal_pos,lg_pos,features,head,dependency_relations,dependencies,misc
0,1,Al,Al,PROPN,NNP,Number=Sing,0,root,NaN,SpaceAfter=No
1,2,-,-,PUNCT,HYPH,NaN,1,punct,NaN,SpaceAfter=No
2,3,Zaman,Zaman,PROPN,NNP,Number=Sing,1,flat,NaN,NaN
3,4,:,:,PUNCT,:,NaN,1,punct,NaN,NaN
4,5,American,american,ADJ,JJ,Degree=Pos,6,amod,NaN,NaN


In [16]:
def make_feature_dict(lst):
    if isinstance(lst, float):
        return {}
    return {feat: value for feat, value in [pair.split('=') for pair in lst]}

In [17]:
feature_dicts = df['features'].str.split('|').apply(make_feature_dict)
features = pd.DataFrame(list(feature_dicts.values))
features.head()

,Case,Definite,Degree,Foreign,Gender,Mood,NumType,Number,Person,Poss,PronType,Reflex,Tense,VerbForm,Voice
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sing,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sing,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,Pos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
data = pd.merge(df, features, left_index=True, right_index=True)
data.drop(['id', 'features', 'head', 'dependency_relations', 'dependencies', 'misc'], axis=1, inplace=True)
data.head()

,form,lemma,universal_pos,lg_pos,Case,Definite,Degree,Foreign,Gender,Mood,NumType,Number,Person,Poss,PronType,Reflex,Tense,VerbForm,Voice
0,Al,Al,PROPN,NNP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sing,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-,-,PUNCT,HYPH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Zaman,Zaman,PROPN,NNP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sing,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,:,:,PUNCT,:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,American,american,ADJ,JJ,NaN,NaN,Pos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
path = 'UD_English/en-ud-features.csv'
data.to_csv(path, index=False)